In [2]:
import tensorflow as tf
import numpy as np
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
import keras
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout)
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, Reshape
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pydub import AudioSegment
import shutil
from keras.preprocessing.image import ImageDataGenerator
import random

In [3]:
# On récupère le dataset créé pour le modele 2D

genres = 'blues classical country disco pop hiphop jazz metal reggae rock'

genres = genres.split()


In [4]:
def GenreModel(input_shape = (288,432,4),classes=10):
    model = keras.Sequential(
        [
            layers.Conv1D(filters=128,
                                  kernel_size=3,
                                  activation='relu',
                                  input_shape=input_shape[1:],
                                  name = 'conv1'),

            layers.MaxPooling1D(name='max1'),

            layers.Conv1D(filters=64,
                                  kernel_size=3,
                                  activation='relu',
                                  name='conv2'),

            layers.MaxPooling1D(name='max2'),   

            layers.Flatten(name='flatten'),

            layers.Dense(100, activation='relu', name='dense1'),
            layers.Dropout(0.5, name='dropout2'),
            layers.Dense(20, activation='relu', name='dense2'),
            layers.Dropout(0.5, name='dropout3'),
            layers.Dense(8, name='dense3')                  
        ]
    )
    
    return model

In [5]:
train_dir = "/content/gdrive/My Drive/spectrograms3sec/train/"
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(288,432),color_mode="rgba",class_mode='categorical',batch_size=128)

validation_dir = "/content/gdrive/My Drive/spectrograms3sec/test/"
vali_datagen = ImageDataGenerator(rescale=1./255)
vali_generator = vali_datagen.flow_from_directory(validation_dir,target_size=(288,432),color_mode='rgba',class_mode='categorical',batch_size=128)



Found 9000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [6]:

model = GenreModel(input_shape=(288,432,4),classes=10)
opt = Adam(learning_rate=0.0005)
model.compile(optimizer=opt,
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

model.summary()




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv1D)               (None, 430, 128)          1664      
_________________________________________________________________
max1 (MaxPooling1D)          (None, 215, 128)          0         
_________________________________________________________________
conv2 (Conv1D)               (None, 213, 64)           24640     
_________________________________________________________________
max2 (MaxPooling1D)          (None, 106, 64)           0         
_________________________________________________________________
flatten (Flatten)            (None, 6784)              0         
_________________________________________________________________
dense1 (Dense)               (None, 100)               678500    
_________________________________________________________________
dropout2 (Dropout)           (None, 100)               0

In [7]:
model.fit(train_generator,epochs=1,validation_data=vali_generator)


ValueError: in user code:

    C:\Users\evebo\anaconda3\lib\site-packages\keras\engine\training.py:830 train_function  *
        return step_function(self, iterator)
    C:\Users\evebo\anaconda3\lib\site-packages\keras\engine\training.py:813 run_step  *
        outputs = model.train_step(data)
    C:\Users\evebo\anaconda3\lib\site-packages\keras\engine\training.py:770 train_step  *
        y_pred = self(x, training=True)
    C:\Users\evebo\anaconda3\lib\site-packages\keras\engine\base_layer.py:989 __call__  *
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    C:\Users\evebo\anaconda3\lib\site-packages\keras\engine\sequential.py:375 call  *
        return super(Sequential, self).call(inputs, training=training, mask=mask)
    C:\Users\evebo\anaconda3\lib\site-packages\keras\engine\functional.py:416 call  *
        inputs, training=training, mask=mask)
    C:\Users\evebo\anaconda3\lib\site-packages\keras\engine\functional.py:551 _run_internal_graph  *
  